In [ ]:
import numpy as np
import requests
import pandas as pd
from datetime import datetime
import mysql.connector
import schedule
import time

In [ ]:
def job():
    print("I'm working...")
    
schedule.every(2).seconds.do(job)

# schedule.every(10).minutes.do(job)
# schedule.every().hour.do(job)
# schedule.every().day.at("10:30").do(job)
# schedule.every(5).to(10).minutes.do(job)
# schedule.every().monday.do(job)
# schedule.every().wednesday.at("13:15").do(job)
# schedule.every().minute.at(":17").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)
#sometimes the job runs more than once (???)

In [ ]:
config = {
  'user': 'newuser',
  'password': 'newpassword',
  'host': '0.0.0.0',
  'database': 'test',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cnx.close()

In [ ]:
#get data through API
with open("app_token.txt") as file: app_token = file.readline().rstrip()

# HTTPS address to access the 311 data
url = "https://data.cityofnewyork.us/resource/erm2-nwe9.json?"
querry = "$where=created_date between '2019-11-02T00:00:00' and '2019-11-09T00:00:00'&$limit=50000"
querry_url = url + "$$app_token=" + app_token + "&" + querry
response = requests.get(querry_url)
calls_311 = response.json()

In [ ]:
#Select keys and extract appropriate data

keys_311 = ['unique_key', 
            'created_date',
            'closed_date',
            'agency',
            'agency_name',
            'complaint_type',
            'descriptor',
            'incident_zip',
            'borough',
           'facility_type',
           'location_type',
            'status',
           'due_date',
            'resolution_action_updated_date',
            'x_coordinate_state_plane',
            'y_coordinate_state_plane',
            'open_data_channel_type',
            'latitude',
            'longitude']

data_311 = []
for call in calls_311:
    dict_calls = {'unique_key': call['unique_key']}
    for key in keys_311:
       if np.sum([i==key for i in call.keys()])==1: 
           dict_calls[key] = call[key]
       else:
            dict_calls[key] = None
    data_311.append(dict_calls)

In [ ]:
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

calls_sql = (
   "insert into 311_calls"
   "(unique_key, created_date, closed_date, agency, agency_name, complaint_type,"
            "descriptor, location_type, incident_zip, borough, facility_type,"
            "status, due_date, resolution_action_updated_date, x_coordinate_state_plane,"
            "y_coordinate_state_plane, open_data_channel_type, latitude, longitude)"
   "values (%(unique_key)s, %(created_date)s, %(closed_date)s, %(agency)s, %(agency_name)s, %(complaint_type)s,"
            "%(descriptor)s, %(location_type)s, %(incident_zip)s, %(borough)s, %(facility_type)s,"
            "%(status)s, %(due_date)s, %(resolution_action_updated_date)s, %(x_coordinate_state_plane)s,"
            "%(y_coordinate_state_plane)s, %(open_data_channel_type)s, %(latitude)s, %(longitude)s)")

for call in data_311:
   cursor.execute(calls_sql, call)

cnx.commit()
cursor.close()
cnx.close()